In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

import os, sys

import fire
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
import os
import json
import re
import string
import pandas as pd
import numpy as np

In [4]:
#Helper function to scrape websites

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [5]:
#Pass the url
url= 'https://www.okayafrica.com/top-15-nigerian-political-twitter-accounts'
response = simple_get(url)

In [6]:
#find all'class_':'rm-shortcode' in the website as our targets are in embeded images and store as lists
resUrl = get_elements(response, search={'find_all':{'class_':'rm-shortcode'}})
resUrl[3]

findaing all of {'class_': 'rm-shortcode'}


<blockquote class="twitter-tweet twitter-custom-tweet" data-partner="rebelmouse" data-twitter-tweet-id="580358893961895936"><div style="margin:1em 0">You can't get a 4-year mandate, do rubbish for 4 years, become desperate for one month and expect to be returned. Na home you dey go</div> — JJ. Omojuwa (@JJ. Omojuwa)<a href="https://twitter.com/Omojuwa/statuses/580358893961895936">1427203317.0</a></blockquote>

In [7]:
#Split by '<blockquote class=' and turn the list to a dataframe
resUrl1 = pd.DataFrame(str(resUrl).split('<blockquote class='), columns=['shortcode']) ; resUrl1

,shortcode
0,[
1,"""twitter-tweet twitter-custom-tweet"" data-part..."
2,"""twitter-tweet twitter-custom-tweet"" data-part..."
3,"""twitter-tweet twitter-custom-tweet"" data-part..."
4,"""twitter-tweet twitter-custom-tweet"" data-part..."
5,"""twitter-tweet twitter-custom-tweet"" data-part..."
6,"""twitter-tweet twitter-custom-tweet"" data-part..."
7,"""twitter-tweet twitter-custom-tweet"" data-part..."
8,"""twitter-tweet twitter-custom-tweet"" data-part..."
9,"""twitter-tweet twitter-custom-tweet"" data-part..."


In [8]:
#Index all strings that contains @
resUrl1 = resUrl1[resUrl1.shortcode.str.contains('@')]
resUrl1.reset_index(drop = True, inplace= True)

In [9]:
resUrl1.shortcode[3]

'"twitter-tweet twitter-custom-tweet" data-partner="rebelmouse" data-twitter-tweet-id="580358893961895936"><div style="margin:1em 0">You can\'t get a 4-year mandate, do rubbish for 4 years, become desperate for one month and expect to be returned. Na home you dey go</div> — JJ. Omojuwa (@JJ. Omojuwa)<a href="https://twitter.com/Omojuwa/statuses/580358893961895936">1427203317.0</a></blockquote>, '

In [10]:
#clean strings using regex and return desired output
resUrl1['user'] = resUrl1.shortcode.apply(lambda x: re.compile('\</div\>(.*?)\<a').search(x).group(1))
resUrl1['ScreenName'] = resUrl1.user.apply(lambda x: re.sub('\((.*?)\)','',x).strip('— '))
resUrl1['GetTag'] = resUrl1.shortcode.apply(lambda x: x.split('href="https://twitter.com/')[1].split('/')[0])

In [11]:
resUrl1[['GetTag', 'ScreenName']]

,GetTag,ScreenName
0,MssZeeUsman,Zainab Usman
1,MssZeeUsman,Zainab Usman
2,MssZeeUsman,Zainab Usman
3,Omojuwa,JJ. Omojuwa
4,gimbakakanda,Gimba Kakanda
5,AishaYesufu,Aisha Yesufu
6,ayosogunro,AYO SOGUNRO
7,MrAyeDee,Mr. Aye Dee
8,MrAyeDee,Mr. Aye Dee
9,gbengasesan,'Gbénga Sèsan


In [12]:
resUrl1 = resUrl1[['GetTag', 'ScreenName']]

In [13]:
#Create an empty dataframe to store all handles and username scrapped
nigcom = pd.DataFrame(columns = ['name', 'handle', 'domain'])

In [14]:
nigcom.name = resUrl1.ScreenName.unique()
nigcom.handle = resUrl1.GetTag.unique()
nigcom['domain'] = 'politics'

In [15]:
### website 2
res = get_elements('https://smartgeek.com.ng/seun/meet-the-2019-top-10-twitter-influencers-in-nigeria', tag = 'h5')
res

['Subscription Form',
 'Facebook Fanpage',
 'RSS feed',
 'Recent Comments',
 'Muhammadu Buhari (@mbuhari)',
 'Dr Dipo Awojide (@OgbeniDipo)',
 'Segalink (@segalink)',
 '@',
 'Four-Eyed Edo Boy (@Alex_Houseof308)',
 'Atiku Abubakar (@atiku)',
 'Davido (@davido)',
 'Reno Omokri (@renoomokri)',
 'Valking (@_valkiing)',
 'Big Brother Naija (@bbnaija)',
 'Tags:',
 'Oluwaseun Sanni',
 'You May Also Like',
 'Best Ways for Freelancers to Make Money Online',
 'Computers Can Now Identify You Through Your Dance',
 'How Private Is Your Data On Social Media Apps?',
 'Social Links',
 'Categories',
 'Archives',
 'Instagram',
 'Twitter']

In [16]:
#Create a new dataframe and store the list
df = pd.DataFrame(res[4:-11], columns = ['name'])
df = df.drop(3, axis = 0)

In [17]:
#Extract the handles and the username
df['handle'] = df.name.apply(lambda x: re.compile(r'\((.*?)\)').findall(x)[0])
df.ScreenName = df.name.apply(lambda x: re.sub('\((.*?)\)','',x))

In [18]:
df['domain'] = ['politics', 'politics','politics','entertainment','politics',
                'entertainment','politics','entertainment','entertainment']

In [19]:
df

,name,handle,domain
0,Muhammadu Buhari (@mbuhari),@mbuhari,politics
1,Dr Dipo Awojide (@OgbeniDipo),@OgbeniDipo,politics
2,Segalink (@segalink),@segalink,politics
4,Four-Eyed Edo Boy (@Alex_Houseof308),@Alex_Houseof308,entertainment
5,Atiku Abubakar (@atiku),@atiku,politics
6,Davido (@davido),@davido,entertainment
7,Reno Omokri (@renoomokri),@renoomokri,politics
8,Valking (@_valkiing),@_valkiing,entertainment
9,Big Brother Naija (@bbnaija),@bbnaija,entertainment


In [20]:
nigcom = nigcom.append(df) ; nigcom.tail(10)

,name,handle,domain
14,Demola Olarewaju,DemolaRewaju,politics
0,Muhammadu Buhari (@mbuhari),@mbuhari,politics
1,Dr Dipo Awojide (@OgbeniDipo),@OgbeniDipo,politics
2,Segalink (@segalink),@segalink,politics
4,Four-Eyed Edo Boy (@Alex_Houseof308),@Alex_Houseof308,entertainment
5,Atiku Abubakar (@atiku),@atiku,politics
6,Davido (@davido),@davido,entertainment
7,Reno Omokri (@renoomokri),@renoomokri,politics
8,Valking (@_valkiing),@_valkiing,entertainment
9,Big Brother Naija (@bbnaija),@bbnaija,entertainment


Manually search for handles on twitter and add to the dataframe

In [21]:
l1 = ['Michael Collins @DONJAZZY', 'Nasir El-Rufai @Elrufai', 'Ali Baba Akporobome @ALIBABAGCFR', 'Ay Comedian @AYCOMEDIAN']
temp1 = pd.DataFrame(l1, columns = ['name'])
temp1['handle'] = temp1.name.apply(lambda x: re.findall('[@]\S*',x)[0])
temp1['domain'] = ['entertainment', 'politics','entertainment','entertainment']
nigcom = nigcom.append(temp1)

In [22]:
l2 = ['Tiwa Savage @TiwaSavage', 'Linda Ikeji @lindaikeji', 'Omotola Jolade @Realomosexy', 'Bella Naija @bellanaija', 'Dbanj Kokmaster @iamdbanj', 'ItweetNaija @Itweetnaija', '9ja Olofofo @9ja_olofofo', 'Olufunke Akindele @funkeakindele', 'Mo Abudu @Moabudu', 'Kemi Adetiba @kemiadetiba', 'Jude Abaga @MI_Abaga','Mercy Eke @real_mercyeke', 'TACHA🔱🇳🇬🇬🇭 @Symply_Tacha','Laycon @Itslaycon']
temp2 = pd.DataFrame(l2, columns = ['name'])
temp2['handle'] = temp2.name.apply(lambda x: re.findall('[@]\S*',x)[0])
temp2['domain'] = ['entertainment']*temp2.shape[0]
nigcom = nigcom.append(temp2)

In [23]:
l3 = ['Stears Business @StearsBusiness', 'Osaretin Victor Asemota @asemota', 'Central Bank of Nigeria @cenbank', 'BudgIT Nigeria @BudgITng', 'Financial Nigeria @FinancialNG', 'Chuba Ezekwesili @ChubaEzeks', 'Preston Timeyin Ideh @TimeyinPI', 'APC Nigeria @OfficialAPCNg', 'Official PDP Nigeria @OfficialPDPNig', "Garba Shehu @GarShehu"]
temp3 = pd.DataFrame(l3, columns = ['name'])
temp3['handle'] = temp3.name.apply(lambda x: re.findall('[@]\S*',x)[0])

lst = ['finance']*(temp3.shape[0]-3)
lst.extend(['politics']*3)
temp3['domain'] = lst
nigcom = nigcom.append(temp3)

In [24]:
nigcom.reset_index(drop = True, inplace = True) ; nigcom

,name,handle,domain
0,Zainab Usman,MssZeeUsman,politics
1,JJ. Omojuwa,Omojuwa,politics
2,Gimba Kakanda,gimbakakanda,politics
3,Aisha Yesufu,AishaYesufu,politics
4,AYO SOGUNRO,ayosogunro,politics
5,Mr. Aye Dee,MrAyeDee,politics
6,'Gbénga Sèsan,gbengasesan,politics
7,The Godfather,Ayourb,politics
8,Khaleesi,AbangMercy,politics
9,Chidi Okereke,Chydee,politics


In [25]:
nigcom.to_csv('data/baselist.csv', index = False)